In [1]:
import pandas as pd
import numpy as np
import re
import sys, os
sys.path.append(os.path.abspath('../containment/'))
#from containment.acaps_functions import *


source_data = pd.read_csv("../../../data/epidemicforecasting/20200416-acaps-covid-19-goverment-measures-dataset-v8.xlsx - Database.csv")

from acaps_functions import *
source_data=tidy_acaps_data(source_data)

In [2]:
for c in source_data.CATEGORY.unique():
    print(c)
    print(source_data.loc[source_data.CATEGORY==c,'MEASURE'].value_counts())
    print("\n")

Public health measures
Strengthening the public health system                488
Introduction of isolation and quarantine policies     394
Awareness campaigns                                   218
General recommendations                               217
Health screenings in airports and border crossings    171
Testing policy                                         60
Other public health measures enforced                  58
Psychological assistance and medical social work       32
Requirement to wear protective gear in public          30
Amendments to funeral and burial regulations           14
Mass population testing                                 8
strengthening the public health system                  7
testing policy                                          1
Name: MEASURE, dtype: int64


Governance and socio-economic measures
Economic measures                                               640
Emergency administrative structures activated or established    176
State of emergency

let's just grab the ACAPS categories with more than 100 entries, excluding government measures

In [14]:
category_exclude_list = source_data.CATEGORY=='Governance and socio-economic measures'
measure_counts = source_data.loc[category_exclude_list==False,'MEASURE'].value_counts()
measures_to_include = measure_counts.index[measure_counts>=10]

In [16]:
measures_to_include

Index(['Strengthening the public health system', 'Limit public gatherings',
       'Introduction of isolation and quarantine policies',
       'Public services closure', 'International flights suspension',
       'Border closure', 'Visa restrictions', 'Awareness campaigns',
       'General recommendations', 'Domestic travel restrictions',
       'Schools closure', 'Partial lockdown',
       'Health screenings in airports and border crossings', 'Curfews',
       'Surveillance and monitoring', 'Testing policy',
       'Other public health measures enforced', 'Full lockdown',
       'Changes in prison-related policies',
       'Additional health/documents requirements upon arrival',
       'Border checks', 'Psychological assistance and medical social work',
       'Requirement to wear protective gear in public',
       'Checkpoints within the country',
       'Amendments to funeral and burial regulations',
       'Lockdown of refugee/idp camps or other minorities'],
      dtype='object')

In [18]:
source_data_trimmed = source_data.loc[source_data.MEASURE.isin(measures_to_include),:]

In [25]:
source_data

,ID,COUNTRY,ISO,ADMIN_LEVEL_NAME,PCODE,REGION,LOG_TYPE,CATEGORY,MEASURE,TARGETED_POP_GROUP,COMMENTS,NON_COMPLIANCE,DATE_IMPLEMENTED,SOURCE,SOURCE_TYPE,LINK,ENTRY_DATE,Alternative source,ADMIN_LEVEL_NAME_MERGEABLE
0,1,Afghanistan,AFG,NaN,NaN,Asia,Introduction / extension of measures,Public health measures,Health screenings in airports and border cross...,No,NaN,NaN,12/02/2020,Ministry of Health,Government,https://moph.gov.af/en/moph-held-emergency-mee...,14/03/2020,NaN,
1,2,Afghanistan,AFG,Kabul,NaN,Asia,Introduction / extension of measures,Public health measures,Introduction of isolation and quarantine policies,No,NaN,NaN,12/02/2020,Ministry of Health,Government,https://moph.gov.af/en/moph-held-emergency-mee...,14/03/2020,NaN,Kabul
2,3,Afghanistan,AFG,NaN,NaN,Asia,Introduction / extension of measures,Public health measures,Awareness campaigns,No,NaN,NaN,12/02/2020,Ministry of Health,Government,https://moph.gov.af/en/moph-held-emergency-mee...,14/03/2020,NaN,
3,4,Afghanistan,AFG,NaN,NaN,Asia,Introduction / extension of measures,Governance and socio-economic measures,Emergency administrative structures activated ...,No,NaN,NaN,12/02/2020,Ministry of Health,Government,https://moph.gov.af/en/moph-held-emergency-mee...,14/03/2020,NaN,
4,5,Afghanistan,AFG,NaN,NaN,Asia,Introduction / extension of measures,Social distancing,Limit public gatherings,No,Nevruz festival cancelled,NaN,12/03/2020,AA,Media,https://www.aa.com.tr/en/asia-pacific/coronavi...,14/03/2020,NaN,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5545,2956,Zimbabwe,ZWE,NaN,NaN,Africa,Introduction / extension of measures,Lockdown,Partial lockdown,No,Zimbabwe will go into lockdown for three weeks...,Arrest/Detention,30/03/2020,Africa News,Media,https://www.africanews.com/2020/03/27/coronavi...,28/03/2020,https://www.bbc.com/news/topics/crr7mlg0rpvt/z...,
5546,4303,Zimbabwe,ZWE,NaN,NaN,Africa,Phase-out measure,Public health measures,Other public health measures enforced,No,Zimbabwe reversed a ban on the sale of alcohol...,Not applicable,03/04/2020,BBC,Media,https://www.bbc.com/news/topics/crr7mlg0rpvt/z...,07/04/2020,NaN,
5547,4646,Zimbabwe,ZWE,NaN,NaN,Africa,Introduction / extension of measures,Governance and socio-economic measures,State of emergency declared,No,"Declaration of State of Disaster, 17 March",Not applicable,17/03/2020,ONHCR Document,UN,See ACAPS folder,10/04/2020,NaN,
5548,5420,Zimbabwe,ZWE,NaN,NaN,Africa,Introduction / extension of measures,Public health measures,Testing policy,No,Zimbabwe begins mass testing with the aim to e...,Not applicable,15/04/2020,BBC,Media,https://www.bbc.com/news/world/africa/live,15/04/2020,NaN,


In [24]:
source_data_trimmed['Dummy']=1.0

source_data_trimmed.pivot(['ISO','DATE_IMPLEMENTED'],'MEASURE','Dummy')

/Users/benjaminsmith/anaconda/envs/pymc3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


ValueError: Length of passed values is 4286, index implies 2.

In [23]:
source_data_trimmed

,ID,COUNTRY,ISO,ADMIN_LEVEL_NAME,PCODE,REGION,LOG_TYPE,CATEGORY,MEASURE,TARGETED_POP_GROUP,COMMENTS,NON_COMPLIANCE,DATE_IMPLEMENTED,SOURCE,SOURCE_TYPE,LINK,ENTRY_DATE,Alternative source,ADMIN_LEVEL_NAME_MERGEABLE,Dummy
0,1,Afghanistan,AFG,NaN,NaN,Asia,Introduction / extension of measures,Public health measures,Health screenings in airports and border cross...,No,NaN,NaN,12/02/2020,Ministry of Health,Government,https://moph.gov.af/en/moph-held-emergency-mee...,14/03/2020,NaN,,1.0
1,2,Afghanistan,AFG,Kabul,NaN,Asia,Introduction / extension of measures,Public health measures,Introduction of isolation and quarantine policies,No,NaN,NaN,12/02/2020,Ministry of Health,Government,https://moph.gov.af/en/moph-held-emergency-mee...,14/03/2020,NaN,Kabul,1.0
2,3,Afghanistan,AFG,NaN,NaN,Asia,Introduction / extension of measures,Public health measures,Awareness campaigns,No,NaN,NaN,12/02/2020,Ministry of Health,Government,https://moph.gov.af/en/moph-held-emergency-mee...,14/03/2020,NaN,,1.0
4,5,Afghanistan,AFG,NaN,NaN,Asia,Introduction / extension of measures,Social distancing,Limit public gatherings,No,Nevruz festival cancelled,NaN,12/03/2020,AA,Media,https://www.aa.com.tr/en/asia-pacific/coronavi...,14/03/2020,NaN,,1.0
5,6,Afghanistan,AFG,NaN,NaN,Asia,Introduction / extension of measures,Movement restrictions,Border closure,No,Closure of borders with Iran,NaN,24/02/2020,Guardian,Media,https://www.theguardian.com/global-development...,14/03/2020,NaN,,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5543,1961,Zimbabwe,ZWE,NaN,NaN,Africa,Introduction / extension of measures,Public health measures,Health screenings in airports and border cross...,No,All our returning residents will be subjected ...,NaN,24/03/2020,The Herald,Media,https://www.herald.co.zw/new-measures-to-stop-...,24/03/2020,NaN,,1.0
5544,1962,Zimbabwe,ZWE,NaN,NaN,Africa,Introduction / extension of measures,Social distancing,Schools closure,No,Zimbabwe closes all schools.,NaN,24/03/2020,News Day,Media,https://www.newsday.co.zw/2020/03/ed-orders-sc...,24/03/2020,NaN,,1.0
5545,2956,Zimbabwe,ZWE,NaN,NaN,Africa,Introduction / extension of measures,Lockdown,Partial lockdown,No,Zimbabwe will go into lockdown for three weeks...,Arrest/Detention,30/03/2020,Africa News,Media,https://www.africanews.com/2020/03/27/coronavi...,28/03/2020,https://www.bbc.com/news/topics/crr7mlg0rpvt/z...,,1.0
5546,4303,Zimbabwe,ZWE,NaN,NaN,Africa,Phase-out measure,Public health measures,Other public health measures enforced,No,Zimbabwe reversed a ban on the sale of alcohol...,Not applicable,03/04/2020,BBC,Media,https://www.bbc.com/news/topics/crr7mlg0rpvt/z...,07/04/2020,NaN,,1.0
